In [1]:
import pandas as pd
from arango import ArangoClient

In [2]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [3]:
pregel = db.pregel

In [4]:
job_id = db.pregel.create_job(
    graph='organization_organization_transaction',
    algorithm='linerank',
    store=False,
    max_gss=None,
    thread_count=1,
    async_mode=False,
    result_field='score'
    # algorithm_params={'threshold': 0.000001}
)

In [5]:
job = pregel.job(job_id)

In [6]:
aql_query = """
FOR res IN PREGEL_RESULT(@job_id)
    RETURN {"_key": res._key, "line_rank": ROUND(res.score*100)/100}
"""
cursor = db.aql.execute(aql_query, bind_vars={'job_id': job_id})
scores = list(cursor)

In [7]:
if db.has_collection('organization_organization_transaction_betweeness'):
    collection = db.collection('organization_organization_transaction_betweeness')
else:
    collection = db.create_collection('organization_organization_transaction_betweeness')

In [8]:
collection.insert_many(scores, overwrite_mode="replace", silent=True)

True

In [10]:
scores_df = pd.DataFrame(scores)

In [18]:
scores_df[scores_df.line_rank > 0.01]

,_key,line_rank
532,54b1e8a8922713623c4dc0f3bdbd29b5,0.02
768,66ebf2a3aa71c90b4df3fc93590e22db,0.17
794,2c398f54dacfc67b9fbe414565285dba,0.02
921,4174ce1ae09c87cce8a4accaf940bc58,0.02
1807,48541ba40bcab069aa5d91c12a5299d0,0.08
